In [10]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import r2_score,mean_absolute_error, mean_squared_error

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import GRU
from tensorflow.keras.layers import LeakyReLU

import keras_tuner
from kerastuner.tuners import RandomSearch
from tensorflow.keras.callbacks import EarlyStopping, Callback

!pip install shap
import shap
import pickle
import copy

import seaborn as sns; sns.set_theme()
from matplotlib import rcParams
import matplotlib.pyplot as plt
import matplotlib as mlp

import lime
import lime.lime_tabular

os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.


In [3]:
inp = pd.read_csv("/kaggle/input/a1101-data/A1101.csv", header = 0, low_memory=False, sep=",")
inp.index = range(len(inp))
inp.head()

inp = inp.groupby('sequence', as_index=False).mean()
inp

,sequence,intensity
0,AAAATSAGTR,0.253411
1,AAASSLLYK,0.748583
2,AAATSAGTR,0.231444
3,AAATSAGTRR,0.341356
4,AAAYFVGYLK,1.000000
...,...,...
6278,YYKDDISYF,0.084687
6279,YYLEKANKI,0.084687
6280,YYPEDPVKL,0.084687
6281,YYQLCQHLK,0.469617


In [4]:
#Tạo list các axit amin để encoding
peptides = inp['sequence'].tolist()
peptides.sort()
vocab = set(''.join([str(i) for i in peptides]))
vocab.add('END')
vocab_list = list(vocab)
vocab_list.sort()
#vocab_list

#Thêm END vào các peptides dài 8-9 có độ dài 10 
char_index = dict((vocab_list[i], i) for i in range(len(vocab_list)))
X = []
x_name = [str(i)[0:10] for i in peptides]
for i in x_name:
    tmp = [char_index[j] for j in str(i)]
    for k in range(0,10 - len(str(i))):
        tmp.append(char_index["END"])
    X.append(tmp)
#X

#Tạo list chứa toàn bộ giá trị CCS
Y = []
for index, row in inp.iterrows():
    my_list = [row.intensity]
    Y.append(my_list)
#all_int_list[0]

X = np.asarray(X)
Y = np.asarray(Y)

In [5]:
#Chia thành tập train, test, val
x_train_all, x_test, y_train_all, y_test = train_test_split(X,Y, test_size=0.1, random_state=42)
x_train, x_val, y_train, y_val = train_test_split(x_train_all, y_train_all, test_size=0.2, random_state=42)

In [ ]:
np.savetxt('CCS_x_test.txt', x_test)
np.savetxt('CCS_y_test.txt', y_test)
np.savetxt('CCS_x_train.txt', x_train)
np.savetxt('CCS_y_train.txt', y_train)
np.savetxt('CCS_x_val.txt', x_val)
np.savetxt('CCS_y_val.txt', y_val)

In [7]:
x_train = np.loadtxt('/kaggle/input/a1101-data/A1101_x_train.txt')

x_test = np.loadtxt('/kaggle/input/a1101-data/A1101_x_test.txt')

y_train = np.loadtxt('/kaggle/input/a1101-data/A1101_y_train.txt')

y_test = np.loadtxt('/kaggle/input/a1101-data/A1101_y_test.txt')

x_val = np.loadtxt('/kaggle/input/a1101-data/A1101_x_val.txt')

y_val = np.loadtxt('/kaggle/input/a1101-data/A1101_y_val.txt')

In [11]:
%%time

#Model LSTM 
model = Sequential()
model.add(Embedding(output_dim = 50, input_dim = 21, input_length = 10))

model.add(LSTM(128,return_sequences=True, input_shape=(10,21)))
model.add(Dropout(0.5333324976566283))
model.add(LSTM(128, return_sequences=False))
model.add(Dropout(0.5630981705545706))

model.add(Dense(64))
model.add(LeakyReLU(alpha=0.3))
model.add(Dropout(0.7995520644054326))

model.add(Dense(1))

optimizermodel = tf.keras.optimizers.Adam(0.001)
optimizermodel.learning_rate.assign(0.01)
model.compile(loss='mse', optimizer = optimizermodel, metrics=['mse'])

hist = model.fit(x_train, y_train,
                batch_size = 5000,
                epochs = 200,
                validation_data = (x_val, y_val))



Epoch 1/200


Argument `alpha` is deprecated. Use `negative_slope` instead.


1/1 ━━━━━━━━━━━━━━━━━━━━ 6s 6s/step - loss: 0.1785 - mse: 0.1785 - val_loss: 0.2907 - val_mse: 0.2907
Epoch 2/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 92ms/step - loss: 0.4140 - mse: 0.4140 - val_loss: 0.2718 - val_mse: 0.2718
Epoch 3/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step - loss: 0.3385 - mse: 0.3385 - val_loss: 0.2497 - val_mse: 0.2497
Epoch 4/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step - loss: 0.4746 - mse: 0.4746 - val_loss: 0.7117 - val_mse: 0.7117
Epoch 5/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step - loss: 0.8436 - mse: 0.8436 - val_loss: 0.0870 - val_mse: 0.0870
Epoch 6/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step - loss: 0.1410 - mse: 0.1410 - val_loss: 0.1032 - val_mse: 0.1032
Epoch 7/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step - loss: 0.1773 - mse: 0.1773 - val_loss: 0.0951 - val_mse: 0.0951
Epoch 8/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step - loss: 0.1235 - mse: 0.1235 - val_loss: 0.1166 - val_mse: 0.1166
Epoch 9/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step - loss: 0.1331 - mse: 0.1331 - val_l

In [13]:
y_pred = model.predict(x_test)
model_r2 = r2_score(y_test, y_pred)
model_mae = mean_absolute_error(y_test, y_pred)
model_mse = mean_squared_error(y_test, y_pred)
print(model_r2, model_mae, model_mse)

20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
0.6434683876737218 0.135180928517858 0.032279353546254975


In [15]:
def build_model(hp):
    model = Sequential()
    model.add(Embedding(output_dim=50, input_dim=21, input_length=10))

    model.add(GRU(hp.Int('units_1', min_value=32, max_value=128, step=32),
                  return_sequences=True, input_shape=(10, 21)))
    model.add(Dropout(hp.Float('dropout_1', min_value=0.0, max_value=0.8)))

    model.add(GRU(hp.Int('units_2', min_value=32, max_value=128, step=32)))
    model.add(Dropout(hp.Float('dropout_2', min_value=0.0, max_value=0.8)))

    model.add(Dense(1))

    learning_rate = hp.Choice('learning_rate', values=[0.001, 0.005, 0.01, 0.05])
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

    model.compile(loss='mse', optimizer=optimizer, metrics=['mse'])

    return model

tuner = RandomSearch(
    build_model,
    objective='val_mse',
    max_trials=30,  
    executions_per_trial=1,  
    directory='tuner_results',
    project_name='gru_model_tuning_9'
)
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

tuner.search(x_train, y_train,
             epochs=200,
             batch_size=tuner.oracle.hyperparameters.Choice('batch_size', values=[64, 128, 256, 512]),
             validation_data=(x_val, y_val),
             callbacks=[early_stopping],
             verbose=1)

best_model = tuner.get_best_models(num_models=1)[0]
best_run = tuner.oracle.get_best_trials(num_trials=1)[0]

best_model.summary()
print(best_run.hyperparameters.values)


Trial 30 Complete [00h 00m 07s]
val_mse: 0.08597074449062347

Best val_mse So Far: 0.02271248959004879
Total elapsed time: 00h 08m 58s


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, 10, 50)         │         1,050 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru (GRU)                       │ (None, 10, 32)         │         8,064 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 10, 32)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_1 (GRU)                     │ (None, 32)             │         6,336 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 15,483 (60.48 KB)

 Trainable params: 15,483 (60.48 KB)

 Non-trainable params: 0 (0.00 B)

{'units_1': 32, 'dropout_1': 0.2810055403004359, 'units_2': 32, 'dropout_2': 0.18264347914316714, 'learning_rate': 0.001, 'batch_size': 512}


In [17]:
%%time

#model GRU
model1 = Sequential()
model1.add(Embedding(output_dim = 50, input_dim = 21, input_length = 10))

model1.add(GRU(32, return_sequences=True))
model1.add(Dropout(0.2810055403004359))

model1.add(GRU(32))
model1.add(Dropout(0.18264347914316714))

model1.add(Dense(1))

optimizer = tf.keras.optimizers.Adam(0.001)

model1.compile(loss='mse', optimizer=optimizer)
model1.fit(x_train, y_train, 
           epochs = 100, 
           batch_size = 512, 
           validation_data = (x_val, y_val))

Epoch 1/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 35ms/step - loss: 0.1326 - val_loss: 0.0863
Epoch 2/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0798 - val_loss: 0.0708
Epoch 3/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0708 - val_loss: 0.0609
Epoch 4/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0634 - val_loss: 0.0548
Epoch 5/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0611 - val_loss: 0.0513
Epoch 6/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0568 - val_loss: 0.0489
Epoch 7/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0536 - val_loss: 0.0469
Epoch 8/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0517 - val_loss: 0.0452
Epoch 9/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0513 - val_loss: 0.0444
Epoch 10/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0500 - val_loss: 0.0441
Epoch 11/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0486 - val_loss: 0.0435
Epoch 12/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0494 - val_loss: 0.043

In [18]:
y_pred1 = model1.predict(x_test)
model1_r2 = r2_score(y_test, y_pred1)
model1_mae = mean_absolute_error(y_test, y_pred1)
model1_mse = mean_squared_error(y_test, y_pred1)
print(model1_r2, model1_mae, model1_mse)

20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
0.6697603089233636 0.12732879526593174 0.02989895811402383


In [19]:
#Chọn model
model = model1

**SHAP**

In [ ]:
%%time

#Tạo 100 cụm dữ liệu để tính toán SHAP
x_trainmeans = shap.kmeans(np.asarray(x_train), 100)
explainer = shap.KernelExplainer(model.predict, x_trainmeans)
shap_values = explainer.shap_values(np.asarray(x_test))

#lưu giá trị SHAP
with open("A1101_shapvalues_GRU.pkl","wb") as f:
    pickle.dump(shap_values, f)

In [22]:
with open("/kaggle/input/a1101-data/shapvaluesA1101.pkl","rb") as f:
    shap_values = pickle.load(f)

**LIME**

In [ ]:
%%time

#Định nghĩa hàm giải thích LIME
explainer = lime.lime_tabular.LimeTabularExplainer(
    training_data = x_train,
    mode = 'regression',
    feature_names = [f'Pos{i+1}' for i in range(10)],
)
positions = [f'Pos{i+1}' for i in range(10)]

lime_values = []
for i in range(len(x_test)):
    x_instance = x_test[i]
    ordered_contributions = {pos: 0.0 for pos in positions}

    #Tính LIME cho từng instance
    exp = explainer.explain_instance(
        data_row = x_instance,
        predict_fn = model.predict
    )

    for feature_value, contribution in exp.as_list():
        pos = next((word for word in feature_value.split() if word.startswith('Pos')), None)
        if pos in ordered_contributions:
            ordered_contributions[pos] += contribution

    contributions_sorted = [ordered_contributions[pos] for pos in positions]
    lime_values.append(contributions_sorted)

with open('A1101_lime_values_LSTM.pkl', 'wb') as f:
    pickle.dump(lime_values, f)

In [24]:
with open('/kaggle/input/a1101-data/A1101_lime_values_LSTM.pkl', 'rb') as f:
    lime_values = pickle.load(f)

**OSA**

In [27]:
%%time

#Tính OSA
baseline = 0
original_output = model.predict(x_test).flatten()
osa_values = np.zeros_like(x_test, dtype=np.float32)

for i in range(10):
    osa_inputs = x_test.copy()
    osa_inputs[:,i] = baseline
    osa_output = model.predict(osa_inputs).flatten()

    osa_values[:,i] = np.abs(original_output - osa_output)



20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
CPU times: user 1.24 s, sys: 81.4 ms, total: 1.32 s
Wall time: 1.08 s


**HEATMAP**

In [ ]:
#Tạo mange lưu các giá trị SHAP/LIME/OSA của 1 axit amin ở cùng 1 vị trí (pos=[1,10])

#Chọn giá trị
explain_values = shap_values

aa = []
i = 0
while i < 10:
    aa.append([[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],[0.0]])
    i+=1

i=0
while i < len(x_test):
    j = 0
    while j < len(x_test[i]):
        aa[j][int(x_test[i][j])].append(explain_values[0][i][j])
        j+=1
    i+=i

heatmap = []

i = 0
while i < len(aa):
    j = 0
    heatmap.append([])
    while j < len(aa[i]):
        if len(aa[i][j]) > 1:
            aa[i][j] = aa[i][j][1:]
            heatmap[i].append(sum(aa[i][j])/float(len(aa[i][j])))
        else:
            heatmap[i].append(0)
        j+=1
    i+=1
heatmap = np.array(heatmap)

In [ ]:
plt.figure()
x_axis_labels = ['A','C','D','E','End','F','G','H','I','K','L','M','N','P','Q','R','S','T','V','W','Y'] # labels for x-axis
y_axis_labels = ['1','2','3','4','5','6','7','8','9','10']
sns.set(font_scale = 2)

rcParams['figure.figsize'] = 20,16
ax = sns.heatmap(heatmap,xticklabels=x_axis_labels, yticklabels=y_axis_labels,linewidths=.5,  cmap="viridis")

ax.set(xlabel='Amino Acid', ylabel='Position', title='Mean SHAP Values - A1101')
ax.figure.savefig('A1101.png')
plt.show()